<h1 style="text-align:center;"><strong>Preparación de los datos<strong></h1>

<h3><strong>Parte 1: Importar las bibliotecas y configurar la ruta</strong></h3>

<h4 style="text-align:justify;">Paso 1: Importar librerías</h4>

In [1]:
import json
import os
import time
import mysql.connector
import pandas as pd
import numpy as np
from ScopusModule import Utils
from sqlalchemy import create_engine
from urllib.parse import quote_plus as url_encode

<h4 style="text-align:justify;">Paso 2: Definir la ruta del directorio</h4>

In [2]:
directorio = "13-07-2022"

<hr>

<hr>

<h3><strong>Parte 2: Preparación de las afiliaciones</strong></h3>

<h4 style="text-align:justify;">Paso 1:Leer las afiliaciones en un dataframe</h4>

In [9]:
dfAffiliations = pd.read_pickle('data/completo/'+directorio+'/affiliations.pkl')

In [10]:
dfAffiliations

,identifier,affiliation_name,city,country
0,112741363,Fundación Sumac Muyu,None,Ecuador
1,125617636,Independent Agricultural Consultant,Santo Domingo,Ecuador
2,127300599,Fundación Agroecológica Río Negro,Santo Domingo de los Tsachilas,Ecuador
3,112632961,Gobierno Provincial de Imbabura,None,Ecuador
4,122274347,EMBIOECSA,Guayaquil,Ecuador
...,...,...,...,...
5367,113849153,Instituto Superior Tecnológico Sucre,Quito,Ecuador
5368,123212418,Asociación Nacional Nutricionistas del Ecuador...,Quito,Ecuador
5369,108669264,Hospital Alli Causai,None,Ecuador
5370,126364744,UNL,Loja,Ecuador


<h4 style="text-align:justify;">Paso 2: Recast de las afiliaciones</h4>

In [11]:
#Establece la columna identifier como índice
dfAffiliations.set_index('identifier', inplace = True)

<h4 style="text-align:justify;">Paso 3: Guardar dataframe de afiliaciones en un pickle</h4>

In [12]:
dfAffiliations.to_pickle('data/completo/'+directorio+'/recast_affiliation.pkl')

In [13]:
dfAffiliations = pd.read_pickle('data/completo/'+directorio+'/recast_affiliation.pkl')

In [14]:
dfAffiliations

,affiliation_name,city,country
identifier,,,
112741363,Fundación Sumac Muyu,None,Ecuador
125617636,Independent Agricultural Consultant,Santo Domingo,Ecuador
127300599,Fundación Agroecológica Río Negro,Santo Domingo de los Tsachilas,Ecuador
112632961,Gobierno Provincial de Imbabura,None,Ecuador
122274347,EMBIOECSA,Guayaquil,Ecuador
...,...,...,...
113849153,Instituto Superior Tecnológico Sucre,Quito,Ecuador
123212418,Asociación Nacional Nutricionistas del Ecuador...,Quito,Ecuador
108669264,Hospital Alli Causai,None,Ecuador


<h4 style="text-align:justify;">Paso 4: Almacenar los Ids de las afiliaciones ecuatorianas</h4>

In [42]:
affiliationsIds = dfAffiliations.index.values.tolist() 

In [43]:
len(affiliationsIds)

5372

<hr>

<hr>

<h3><strong>Parte 3: Preparacion de los artículos</strong></h3>

<h4 style="text-align:justify;">Paso 1: Leer los artículos en un dataframe</h4>

In [28]:
dfArticles = pd.read_pickle('data/completo/'+directorio+'/article.pkl')

In [32]:
dfArticles

,identifier,title,publication_date,prism:doi,abstract,affiliations,authors,author_keywords,author_count,affiliation_count
0,85133492759,"Metamodeling and Audio Signals Design Process,...",2023-01-01,10.1007/978-3-031-09659-4_42,The context of the following work arises from ...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57781304700', 'authname': 'Peñahe...",Audio signals design process | Experimental de...,3,2
1,85133293730,Facility Layout Design in Textile MSMEs. Liter...,2023-01-01,10.1007/978-3-031-09360-9_23,The capacity to respond and adapt to risks and...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57777188600', 'authname': 'Albán-...",Facility layout problem | Indicators | Resilie...,5,2
2,85132518705,Optimal finite sample post-selection confidenc...,2023-01-01,10.1016/j.jspi.2022.06.001,Uniformly most powerful confidence distributio...,"[{'afid': '60121225', 'affilname': 'Leuven Sta...","[{'authid': '57572669100', 'authname': 'Garcia...",Confidence distribution | Confidence interval ...,2,2
3,85112575431,"Corporate social responsibility, green innovat...",2022-12-19,10.1108/CR-12-2020-0160,Purpose: The purpose of this paper is to explo...,"[{'afid': '60072050', 'affilname': 'Universida...","[{'authid': '57195298226', 'authname': 'Padill...",Competitiveness | Corporate social responsibil...,2,2
4,85109263966,Analysing competing logics towards sustainable...,2022-12-19,10.1108/SCM-07-2020-0354,Purpose: To understand the context surrounding...,"[{'afid': '60112757', 'affilname': 'Excelia Bu...","[{'authid': '57190878471', 'authname': 'León B...",Food industry | Global supply chain | Supply c...,3,3
...,...,...,...,...,...,...,...,...,...,...
40370,37049240084,Sodiro herbarium,1923-01-01,10.1126/science.58.1510.465,NaN,"[{'afid': '60032280', 'affilname': 'United Sta...","[{'authid': '23082286800', 'authname': 'Hitchc...",NaN,1,1
40371,77957209355,The capsulín cherry: A superior form of the no...,1922-01-01,10.1093/oxfordjournals.jhered.a102156,NaN,"[{'afid': '60032280', 'affilname': 'United Sta...","[{'authid': '36511689400', 'authname': 'Popeno...",NaN,2,2
40372,84941812731,Uncinarial nephritis,1921-01-01,10.1001/archinte.1921.00100170051003,NaN,"[{'afid': '60072042', 'affilname': 'Universida...","[{'authid': '56852172200', 'authname': 'Rojas ...",NaN,2,1
40373,84944669127,Hookworm and other intestinal parasites in ecu...,1920-12-18,10.1001/jama.1920.02620510016007,NaN,"[{'afid': '115872020', 'affilname': 'South Ame...","[{'authid': '56910946000', 'authname': 'Royer ...",NaN,1,1


<h4 style="text-align:justify;">Paso 2: Obtener los artículos sin relación</h4>

In [33]:
articlesIdsSinRelacion = pd.read_pickle('data/completo/'+directorio+'/unrelated_articles.pkl')

In [34]:
len(articlesIdsSinRelacion)

423

<h4 style="text-align:justify;">Paso 3: Recast de los artículos</h4>

In [35]:
#Elimina los registros con affiliations igual a NaN
#dfArticles.dropna(subset=['affiliation'], inplace=True)

#Reinicia los índices del dataframe
dfArticles.reset_index(drop=True, inplace=True)

#Renombra las sigueintes columnas
dfArticles.rename(columns = {'prism:doi': 'doi'}, inplace = True)

#Establece la columna identifier como índice
dfArticles.set_index('identifier', inplace = True)

In [36]:
#Elimina los artículos sin relaciones
dfArticles.drop(articlesIdsSinRelacion, inplace=True)

In [37]:
dfArticles

,title,publication_date,doi,abstract,affiliations,authors,author_keywords,author_count,affiliation_count
identifier,,,,,,,,,
85133492759,"Metamodeling and Audio Signals Design Process,...",2023-01-01,10.1007/978-3-031-09659-4_42,The context of the following work arises from ...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57781304700', 'authname': 'Peñahe...",Audio signals design process | Experimental de...,3,2
85133293730,Facility Layout Design in Textile MSMEs. Liter...,2023-01-01,10.1007/978-3-031-09360-9_23,The capacity to respond and adapt to risks and...,"[{'afid': '60106645', 'affilname': 'Universida...","[{'authid': '57777188600', 'authname': 'Albán-...",Facility layout problem | Indicators | Resilie...,5,2
85132518705,Optimal finite sample post-selection confidenc...,2023-01-01,10.1016/j.jspi.2022.06.001,Uniformly most powerful confidence distributio...,"[{'afid': '60121225', 'affilname': 'Leuven Sta...","[{'authid': '57572669100', 'authname': 'Garcia...",Confidence distribution | Confidence interval ...,2,2
85112575431,"Corporate social responsibility, green innovat...",2022-12-19,10.1108/CR-12-2020-0160,Purpose: The purpose of this paper is to explo...,"[{'afid': '60072050', 'affilname': 'Universida...","[{'authid': '57195298226', 'authname': 'Padill...",Competitiveness | Corporate social responsibil...,2,2
85109263966,Analysing competing logics towards sustainable...,2022-12-19,10.1108/SCM-07-2020-0354,Purpose: To understand the context surrounding...,"[{'afid': '60112757', 'affilname': 'Excelia Bu...","[{'authid': '57190878471', 'authname': 'León B...",Food industry | Global supply chain | Supply c...,3,3
...,...,...,...,...,...,...,...,...,...
33646049406,Beiträge zur Kenntnis des Fluges der Fledermäu...,1925-01-01,10.1111/j.1463-6395.1925.tb00265.x,NaN,"[{'afid': '120628571', 'affilname': 'Marbach a...","[{'authid': '35935630900', 'authname': 'Spillm...",NaN,1,1
77957209355,The capsulín cherry: A superior form of the no...,1922-01-01,10.1093/oxfordjournals.jhered.a102156,NaN,"[{'afid': '60032280', 'affilname': 'United Sta...","[{'authid': '36511689400', 'authname': 'Popeno...",NaN,2,2
84941812731,Uncinarial nephritis,1921-01-01,10.1001/archinte.1921.00100170051003,NaN,"[{'afid': '60072042', 'affilname': 'Universida...","[{'authid': '56852172200', 'authname': 'Rojas ...",NaN,2,1


<h4 style="text-align:justify;">Paso 3: Guardar dataframe de artículos en un pickle</h4>

In [38]:
dfArticles.to_pickle('data/completo/'+directorio+'/recast_article.pkl')

In [39]:
dfArticles = pd.read_pickle('data/completo/'+directorio+'/recast_article.pkl')

<hr>

<hr>

<h3><strong>Parte 4: Preparación de los autores ecuatorianos de los artículos</strong></h3>

<h4 style="text-align:justify;">Paso 1: Leer los autores almacenados en un dataframe</h4>

In [53]:
dfAuthors = pd.read_pickle('data/completo/'+directorio+'/authors.pkl')
dfAuthors

,identifier,first_name,last_name,auth_name,initials
0,57219054382,Jorge Edwin,Ormaza Andrade,Ormaza Andrade J.E.,J.E.
1,57192930404,Mario,Hurtado,Hurtado M.,M.
2,57192803433,Ruth Elizabeth,Minga-Vallejo,Minga-Vallejo R.E.,R.E.
3,57220465983,Carlos,Tapia,Tapia C.,C.
4,57215549671,Pelayo,Salinas-DeLeón,Salinas-DeLeón P.,P.
...,...,...,...,...,...
39220,57216415863,Johana,Quinonez,Quinonez J.,J.
39221,57209980941,José,Palacios,Palacios J.,J.
39222,57192640335,Santiago,Aguiar,Aguiar S.,S.
39223,57191111922,J. R.,VELASTEGUI,VELASTEGUI J.,J.R.


<h4 style="text-align:justify;">Paso 5: Recast de los resultados</h4>

In [54]:
#Establece la columna identifier como índice
dfAuthors.set_index('identifier', inplace = True)

In [55]:
dfAuthors

,first_name,last_name,auth_name,initials
identifier,,,,
57219054382,Jorge Edwin,Ormaza Andrade,Ormaza Andrade J.E.,J.E.
57192930404,Mario,Hurtado,Hurtado M.,M.
57192803433,Ruth Elizabeth,Minga-Vallejo,Minga-Vallejo R.E.,R.E.
57220465983,Carlos,Tapia,Tapia C.,C.
57215549671,Pelayo,Salinas-DeLeón,Salinas-DeLeón P.,P.
...,...,...,...,...
57216415863,Johana,Quinonez,Quinonez J.,J.
57209980941,José,Palacios,Palacios J.,J.
57192640335,Santiago,Aguiar,Aguiar S.,S.


<h4 style="text-align:justify;">Paso 6: Guardar dataframe de autores en un pickle</h4>

In [56]:
dfAuthors.to_pickle('data/completo/'+directorio+'/recast_author.pkl')

In [57]:
dfAuthors = pd.read_pickle('data/completo/'+directorio+'/recast_author.pkl')

<hr>

<hr>

<h3><strong>Parte 4: Preparación de las relaciones entre artículo y afiliación</strong></h3>

<h4 style="text-align:justify;">Paso 1: Obtener los datos de las relaciones de artículo y afiliación</h4>

In [74]:
affiliationsIds = dfAffiliations.index.tolist()
dataArticlesAffiliations = []
for index, row in dfArticles.iterrows():
    for affil in row['affiliations']:
        if affil['afid'] in affiliationsIds:
            dataArticlesAffiliations.append({
                'article_id': index,
                'affiliation_id': affil['afid']
            })

In [75]:
print('Número de registros de article_affiliation:', len(dataArticlesAffiliations))

Número de registros de article_affiliation: 55302


<h4 style="text-align:justify;">Paso 2: Cargar los datos de las relaciones entre artículo y afiliación en un dataframe</h4>

In [78]:
dfArticlesAffiliations = pd.DataFrame(dataArticlesAffiliations)

In [79]:
dfArticlesAffiliations

,article_id,affiliation_id
0,85133492759,60106645
1,85133293730,60106645
2,85133293730,60072035
3,85132518705,60072061
4,85112575431,60072050
...,...,...
55297,33646049406,120628571
55298,77957209355,109557708
55299,84941812731,60072042
55300,84944669127,115872020


<h4 style="text-align:justify;">Paso 3: Guardar dataframe de las relaciones entre artículo y afiliación en un pickle</h4>

In [80]:
dfArticlesAffiliations.to_pickle('data/completo/'+directorio+'/recast_article_affiliation.pkl')

In [81]:
dfArticlesAffiliations = pd.read_pickle('data/completo/'+directorio+'/recast_article_affiliation.pkl')

<hr>

<hr>

<h3><strong>Parte 5: Preparación de las relaciones entre artículo y autor</strong></h3>

<h4 style="text-align:justify;">Paso 1: Obtener los datos de las relaciones de artículo y autor</h4>

In [82]:
affiliationsIds = dfAffiliations.index.tolist()
dataArticlesAuthors = []
for index, row in dfArticles.iterrows():
    for author in row['authors']:
        if any(item in affiliationsIds for item in author['afid']):
            dataArticlesAuthors.append({
                'article_id': index,
                'author_id': author['authid']
            })

In [83]:
print('Número de registros de article_author', len(dataArticlesAuthors))

Número de registros de article_author 102592


<h4 style="text-align:justify;">Paso 2: Cargar los datos de las relaciones entre artículo y autor en un dataframe</h4>

In [85]:
dfArticlesAuthors = pd.DataFrame(dataArticlesAuthors)
dfArticlesAuthors.head(5)

,article_id,author_id
0,85133492759,57781304700
1,85133293730,57777188600
2,85133293730,57209969066
3,85133293730,57581708700
4,85133293730,57219597377


<h4 style="text-align:justify;">Paso 3: Guardar dataframe de las relaciones entre artículo y autor en un pickle</h4>

In [86]:
dfArticlesAuthors.to_pickle('data/completo/'+directorio+'/recast_article_author.pkl')

In [87]:
dfArticlesAuthors = pd.read_pickle('data/completo/'+directorio+'/recast_article_author.pkl')

<hr>

<hr>

<h3><strong>Parte 6: Extraer las relaciones entre autor y afiliacion</strong></h3>

<h4 style="text-align:justify;">Paso 1: Obtener los datos de las relaciones entre autor y afiliación</h4>

In [98]:
dataAuthorsAffiliations = []
for index, row in dfArticles.iterrows():
    for author in row['authors']:
        for afid in author['afid']:
            if afid in affiliationsIds:
                dataAuthorsAffiliations.append({
                    'author_id': author['authid'],
                    'affiliation_id': afid
                })

In [99]:
print('Número de registros de autor_affiliation (incluye repetidos): ', len(dataAuthorsAffiliations))

Número de registros de autor_affiliation (incluye repetidos):  110457


<h4 style="text-align:justify;">Paso 2: Eliminar los datos repetidos de las relaciones entre autor y afiliación</h4>

In [100]:
dataAuthorsAffiliations = [eval(t) for t in set([str(d) for d in dataAuthorsAffiliations])]  

In [101]:
print('Número de registros de autor_affiliation: ', len(dataAuthorsAffiliations))

Número de registros de autor_affiliation:  48362


<h4 style="text-align:justify;">Paso 3: Cargar los datos de las relaciones entre autor y afiliación en un dataframe</h4>

In [102]:
dfAuthorsAffiliations = pd.DataFrame(dataAuthorsAffiliations)

In [103]:
dfAuthorsAffiliations

,author_id,affiliation_id
0,57556791300,60072035
1,57221752656,60105694
2,6508191924,101330130
3,56481581500,114889267
4,57218280892,60105694
...,...,...
48357,57201792387,60072036
48358,57204909849,60104598
48359,57205361475,60105693
48360,57226459002,60108339


<h4 style="text-align:justify;">Paso 4: Guardar dataframe de las relaciones entre autor y afiliación en un pickle</h4>

In [104]:
dfAuthorsAffiliations.to_pickle('data/completo/'+directorio+'/recast_author_affiliation.pkl')

In [105]:
dfAuthorsAffiliations = pd.read_pickle('data/completo/'+directorio+'/recast_author_affiliation.pkl')

<hr>

<hr>

<h3><strong>Parte 7: Preparación de las author_keywords de los artículos</strong></h3>

<h4 style="text-align:justify;">Paso 1: Obtener los datos de las author_keywords</h4>

In [59]:
dataAuthorKeywords = []
for index, row in dfArticles.iterrows():
    if type(row['author_keywords']) != float:
        authorKeywords = row['author_keywords'].split(' | ')
        for item in authorKeywords:  
            dataAuthorKeywords.append(item)

In [60]:
len(dataAuthorKeywords)

162829

<h4 style="text-align:justify;">Paso 2: Eliminar los datos repetidos de las author_keywords</h4>

In [61]:
dataAuthorKeywords = list(dict.fromkeys(dataAuthorKeywords))

In [62]:
len(dataAuthorKeywords)

86314

<h4 style="text-align:justify;">Paso 3: Cargar los datos de las author_keywords</h4>

In [63]:
dataAuthorKeywords.index("Papillary carcinoma")

86205

In [64]:
dfAuthorKeywords = pd.DataFrame(dataAuthorKeywords, columns=['name'])

In [65]:
dfAuthorKeywords.index.name = 'identifier'
dfAuthorKeywords

,name
identifier,
0,Audio signals design process
1,Experimental design processes
2,Fictional spaces
3,Sound and changing forms
4,Facility layout problem
...,...
86309,recurrent selection
86310,"3,3-dimethoxy-22,26-epiminocholestane"
86311,Solanum pseudoquina


<h4 style="text-align:justify;">Paso 4: Guardar dataframe de las author_keywords en un pickle</h4>

In [66]:
dfAuthorKeywords.to_pickle('data/completo/'+directorio+'/recast_author_keyword.pkl')

In [67]:
dfAuthorKeywords = pd.read_pickle('data/completo/'+directorio+'/recast_author_keyword.pkl')

<hr>

<hr>

<h3><strong>Parte 7: Preparación de las relaciones entre artículos y author_keywords</strong></h3>

<h4 style="text-align:justify;">Paso 1: Obtener los datos de las relaciones entre artículos y author_keywords</h4>

In [106]:
dataArticlesAuthorKeywords = []
for index, row in dfArticles.iterrows():
    if type(row['author_keywords']) != float:
        authorKeywords = row['author_keywords'].split(' | ')
        for item in authorKeywords:  
            dataArticlesAuthorKeywords.append({
                'article_id': index,
                'author_keyword_id': dataAuthorKeywords.index(item)
            })

In [107]:
print('Número de registros de article_author_keyword: ', len(dataArticlesAuthorKeywords))

Número de registros de article_author_keyword:  162829


<h4 style="text-align:justify;">Paso 2: Cargar los datos de las relaciones entre article y author_keywords en un dataframe</h4>

In [108]:
dfArticlesAuthorKeywords = pd.DataFrame(dataArticlesAuthorKeywords)

In [109]:
dfArticlesAuthorKeywords

,article_id,author_keyword_id
0,85133492759,0
1,85133492759,1
2,85133492759,2
3,85133492759,3
4,85133293730,4
...,...,...
162824,49349128136,34003
162825,49349128136,86311
162826,49349128136,86312
162827,0014703748,86294


<h4 style="text-align:justify;">Paso 4: Guardar dataframe de las relaciones entre article y author_keywords en un pickle</h4>

In [110]:
dfArticlesAuthorKeywords.to_pickle('data/completo/'+directorio+'/recast_article_author_keyword.pkl')

In [111]:
dfArticlesAuthorKeywords = pd.read_pickle('data/completo/'+directorio+'/recast_article_author_keyword.pkl')